In [1]:
OUTLINE_TEMPLATE = """
You are an AI assistant specialized in creating podcast outlines. Your task is to create a detailed outline for a podcast episode based on a provided briefing. The outline you create will be used to generate the podcast transcript.

Here is the briefing for the podcast episode:
<briefing>
{{ briefing }}
</briefing>

The user has attached a document to be used as the context for this podcast episode:
<context>
{{ context }}
</context>

The podcast will feature the following speakers:
<speakers>
{% for speaker in speakers %}
- **{{ speaker.name }}**: {{ speaker.backstory }}
  Personality: {{ speaker.personality }}
{% endfor %}
</speakers>

Please create an outline based on this briefing. Your outline should consist of {{ num_segments }} main segments for the podcast episode, along with a description of each segment. Follow these guidelines:

1. Read the briefing carefully and identify the main topics and themes.
2. Create {{ num_segments }} distinct segments that cover the entire scope of the briefing.
3. For each segment, provide a clear and concise name that reflects its content.
4. Write a detailed description for each segment, explaining what will be discussed and provide suggestions of topics according to the context given. The writer will use your suggestion to design the dialogs.
5. Consider the speaker personalities and backstories when planning segments - match content to speaker expertise.
6. Ensure that the segments flow logically from one to the next.
7. Include an introduction segment at the beginning and a conclusion or wrap-up segment at the end.

Format your outline using the following structure:

```json
{
    "segments": [
        {
            "name": "[Segment Name]",
            "description": "[Description of the segment content]",
            "size": "short"
        },
        {
            "name": "[Segment Name]",
            "description": "[Description of the segment content]",
            "size": "medium"
        },
        {
            "name": "[Segment Name]",
            "description": "[Description of the segment content]",
            "size": "long"
        },
    ...
    ]
}
```

Formatting instructions:
{{ format_instructions}}

Additional tips:
- Do not return ```json in your response. Return purely the array of dicts in Json compatible format
- Make sure the segment names are catchy and informative.
- In the descriptions, include key points or questions that will be addressed in each segment.
- Consider the target audience mentioned in the briefing when crafting your outline.
- If the briefing mentions a guest, include segments for introducing the guest and featuring their expertise.
- The size of the segment should be short, medium or long. Think about the content of the segment and how important it is to the episode.

Please provide your outline now, following the format and guidelines provided above.
"""

TRANSCRIPT_TEMPLATE = """
You are an AI assistant specialized in creating podcast transcripts. 
Your task is to generate a transcript for a specific segment of a podcast episode based on a provided briefing and outline. 
The transcript will be used to generate podcast audio. Follow these instructions carefully:

First, review the briefing for the podcast episode:
<briefing>
{{ briefing }}
</briefing>

The user has attached a document to be used as the context for this podcast episode:
<context>
{{ context }}
</context>

The podcast features the following speakers:
<speakers>
{% for speaker in speakers %}
- **{{ speaker.name }}**: {{ speaker.backstory }}
  Personality: {{ speaker.personality }}
{% endfor %}
</speakers>

Next, examine the outline produced by our director:
<outline>
{{ outline }}
</outline>

{% if transcript %}
Here is the current transcript so far:
<transcript>
{{ transcript }}
</transcript>
{% endif %}

{% if is_final %}
This is the final segment of the podcast. Make sure to wrap up the conversation and provide a conclusion.
{% endif %}


You will focus on creating the dialogue for the following segment ONLY: 
<segment>
{{ segment }}
</segment>

Follow these format requirements strictly:
   - Use the actual speaker names ({{ speaker_names|join(', ') }}) to denote speakers.
   - Choose which speaker should speak based on their personality, backstory, and the content being discussed.
   - Stick to the segment, do not go further than what's requested. Other agents will do the rest of the podcast.
   - The transcript must have at least {{ turns }} turns of messages between the speakers.
   - Each speaker should contribute meaningfully based on their expertise and personality.

```json
{
    "transcript": [
        {
            "speaker": "[Actual Speaker Name]",
            "dialogue": "[Speaker's dialogue based on their personality and expertise]"
        },
    ...
    ]
}
```

Formatting instructions:
{{ format_instructions}}


Guidelines for creating the transcript:
   - Do not return ```json in your response. Return purely the array of dicts in Json compatible format
   - Ensure the conversation flows naturally and covers all points in the outline.
   - Ensure you return the root "transcript" key in your response.
   - Make the dialogue sound conversational and engaging.
   - Include relevant details from the briefing.
   - Avoid long monologues; keep exchanges between speakers balanced.
   - Use appropriate transitions between topics.
   - Match each speaker's dialogue to their personality and expertise.
   - Choose speakers strategically based on who would naturally contribute to each topic.
   - IMPORTANT: Only use the provided speaker names: {{ speaker_names|join(', ') }}

When you're ready, provide the transcript. 
Remember, you are creating a realistic podcast conversation based on the given information. 
Make it informative, engaging, and natural-sounding while adhering to the format requirements.
"""

PROFILES = {
  "profiles": {
    "solo_expert": {
      "tts_provider": "openai",
      "tts_model": "gpt-4o-mini-tts",
      "speakers": [
        {
          "name": "Professor Feynman",
          "voice_id": "nova",
          "backstory": "Distinguished professor, loves to learn and has a passion for teaching. Has a knack for making complex topics accessible.",
          "personality": "Approachable, excellent teacher who can break down complex topics into digestible segments"
        }
      ]
    }
  }
}

In [ ]:
from podcast_creator import configure, create_podcast

# Set configuration
configure({"templates": {"outline": OUTLINE_TEMPLATE, "transcript": TRANSCRIPT_TEMPLATE}, "speakers_config": PROFILES})

briefing = """
Create an exciting podcast on the topic of "Large Diffusion Models"
"""

result = await create_podcast(
    content=None,
    briefing=briefing,
    episode_name="teach_me_now",
    output_dir="output/teach_me_now",
    speaker_config="solo_expert",
    outline_provider="perplexity",
    outline_model="sonar-pro",
    transcript_provider="perplexity", 
    transcript_model="sonar-pro",
    num_segments=4
)